In [32]:
import numpy as np

# Create a synthetic image with dimensions 100 x 100 x 100
small_test_image = np.random.randint(0, 256, (100, 100, 100), dtype=np.uint8)

# Save it to a .raw file
small_test_file = "C:/Users/Abdel Rahman Rashwan/Desktop/Bachelor/Data/small_test_image.raw"
with open(small_test_file, 'wb') as f:
    f.write(small_test_image.tobytes())

print(f"Small test file saved at {small_test_file}")


Small test file saved at C:/Users/Abdel Rahman Rashwan/Desktop/Bachelor/Data/small_test_image.raw


In [33]:
import os
import ctypes

# Read 8-bit .raw image file
def py_p3dReadRaw8(filename, dimx, dimy, dimz=0):
    if not os.path.isfile(filename):
        print("Error: File not found.")
        return None

    expected_size = dimx * dimy * dimz
    actual_size = os.path.getsize(filename)
    if actual_size != expected_size:
        print(f"Error: Expected {expected_size} bytes, but got {actual_size}.")
        return None

    # Allocate memory for image data
    image_data = (ctypes.c_ubyte * expected_size)()
    with open(filename, "rb") as f:
        f.readinto(image_data)

    return image_data

# Write 8-bit .raw image file
def py_p3dWriteRaw8(image_data, filename, dimx, dimy, dimz=0):
    with open(filename, "wb") as f:
        f.write(bytearray(image_data))
    print(f"Image saved to {filename}")


In [5]:
import ctypes
import time
import os

# Define functions to read and write 8-bit RAW images
def py_p3dReadRaw8(filename, dimx, dimy, dimz):
    """
    Reads an 8-bit RAW image and returns it as a ctypes array.
    """
    if not os.path.isfile(filename):
        raise FileNotFoundError(f"File not found: {filename}")
    
    image_data = (ctypes.c_ubyte * (dimx * dimy * dimz))()
    with open(filename, "rb") as file:
        file.readinto(image_data)
    return image_data

def py_p3dWriteRaw8(image_data, filename, dimx, dimy, dimz):
    """
    Writes an 8-bit RAW image from a ctypes array to a file.
    """
    with open(filename, "wb") as file:
        file.write(image_data)

# Load the DLL
skeleton_dll = ctypes.CDLL(r"C:/Users/Abdel Rahman Rashwan/Desktop/BlobAnalysisProject/P3D_Skel.dll")

# Define argument and return types for DLL functions
skeleton_dll.p3dLKCSkeletonization.argtypes = [
    ctypes.POINTER(ctypes.c_ubyte), ctypes.POINTER(ctypes.c_ubyte),
    ctypes.c_int, ctypes.c_int, ctypes.c_int, ctypes.c_void_p
]
skeleton_dll.p3dLKCSkeletonization.restype = ctypes.c_int

skeleton_dll.p3dSimpleSkeletonPruning.argtypes = [
    ctypes.POINTER(ctypes.c_ubyte), ctypes.POINTER(ctypes.c_ubyte),
    ctypes.c_int, ctypes.c_int, ctypes.c_int, ctypes.c_int, ctypes.c_void_p
]
skeleton_dll.p3dSimpleSkeletonPruning.restype = ctypes.c_int

skeleton_dll.p3dSkeletonLabeling.argtypes = [
    ctypes.POINTER(ctypes.c_ubyte), ctypes.POINTER(ctypes.c_ubyte),
    ctypes.c_int, ctypes.c_int, ctypes.c_int, ctypes.c_void_p
]
skeleton_dll.p3dSkeletonLabeling.restype = ctypes.c_int

# Main function to perform skeletonization
def apply_skeletonization(input_file, output_file, dimx, dimy, dimz, skel_thresh):
    # Step 1: Read input file
    print("Reading input image...")
    input_image = py_p3dReadRaw8(input_file, dimx, dimy, dimz)
    
    # Prepare output buffers
    skel_image = (ctypes.c_ubyte * (dimx * dimy * dimz))()
    pruned_image = (ctypes.c_ubyte * (dimx * dimy * dimz))()
    labeled_image = (ctypes.c_ubyte * (dimx * dimy * dimz))()
    
    # Step 2: Apply LKC Skeletonization
    print("Applying LKC Skeletonization...")
    start_time = time.time()
    err_code = skeleton_dll.p3dLKCSkeletonization(
        ctypes.cast(input_image, ctypes.POINTER(ctypes.c_ubyte)),
        ctypes.cast(skel_image, ctypes.POINTER(ctypes.c_ubyte)),
        dimx, dimy, dimz, None
    )
    if err_code != 2:
        raise RuntimeError(f"LKC Skeletonization failed with error code {err_code}")

    # Step 3: Apply Simple Skeleton Pruning
    print("Applying Skeleton Pruning...")
    err_code = skeleton_dll.p3dSimpleSkeletonPruning(
        ctypes.cast(skel_image, ctypes.POINTER(ctypes.c_ubyte)),
        ctypes.cast(pruned_image, ctypes.POINTER(ctypes.c_ubyte)),
        dimx, dimy, dimz, skel_thresh, None
    )
    if err_code != 2:
        raise RuntimeError(f"Skeleton Pruning failed with error code {err_code}")

    # Step 4: Apply Skeleton Labeling
    print("Applying Skeleton Labeling...")
    err_code = skeleton_dll.p3dSkeletonLabeling(
        ctypes.cast(pruned_image, ctypes.POINTER(ctypes.c_ubyte)),
        ctypes.cast(labeled_image, ctypes.POINTER(ctypes.c_ubyte)),
        dimx, dimy, dimz, None
    )
    if err_code != 2:
        raise RuntimeError(f"Skeleton Labeling failed with error code {err_code}")

    end_time = time.time()
    print(f"Skeletonization, pruning, and labeling completed in {end_time - start_time:.3f} seconds.")

    # Step 5: Write output file
    print(f"Writing output image to {output_file}")
    py_p3dWriteRaw8(labeled_image, output_file, dimx, dimy, dimz)
    print("Output image saved successfully.")

# Set paths and parameters based on the screenshot
input_file = "C:\\Users\\Abdel Rahman Rashwan\\Desktop\\Bachelor\\Data\\Output_median_otsu_eroded_dilated.raw"
output_file = "C:\\Users\\Abdel Rahman Rashwan\\Desktop\\Bachelor\\Data\\Output_median_otsu_skeleton_LKC_pruned_labeled.raw"
dimx, dimy, dimz = 700, 700, 700
skel_thresh = 5

# Run the skeletonization process
apply_skeletonization(input_file, output_file, dimx, dimy, dimz, skel_thresh)


Reading input image...
Applying LKC Skeletonization...
Applying Skeleton Pruning...
Applying Skeleton Labeling...
Skeletonization, pruning, and labeling completed in 1839.109 seconds.
Writing output image to C:\Users\Abdel Rahman Rashwan\Desktop\Bachelor\Data\Output_median_otsu_skeleton_LKC_pruned_labeled.raw
Output image saved successfully.
